# Introduction to Financial Engineering<br>
## Assignment #1<br>
### 2018015350 류형진
<br>
<br>

## 코드 설명
```
1. 데이터를 불러온다.

2. 불러온 데이터에서 어느 종목을 가지고 포트폴리오를 구성할지 결정
  2-1. 이번 프로젝트에서는 2010년 1월 KOSPI의 MCAP 상위 10개의 종목을 우선시로 추가하기로 결정
  (만약 없거나 이름이 바뀐 경우에는 그 다음 종목으로 대체하는 등으로 진행)
  2-2. 만약 총 A개의 종목으로 포트폴리오를 구성하고자 한다면, (A - 10) 개의 종목을 누적 수익률이 큰 것부터 선별

3. 선별한 종목들로 최소 분산, 최대 샤프 비율 포트폴리오 구성
  3-1. 최소 분산 포트폴리오는 cxvopt의 quadratic programming을 이용해 Markowitz model을 solve
 (이때 사용하는 r_i는 과거 데이터의 평균, variance 값들은 과거 데이터의 rho를 이용해 GARCH(1,1)로 예측해 공분산 행렬을 구성)
  3-2. 최대 샤프 비율 포트폴리오는, scipy의 optimize를 활용해 solve.

4. 구성한 포트폴리오들의 미래 데이터 1달에 대한 testing을 진행
(Portfolio return을 계산)

5. train time window를 한 달 증가 시켜서, 1달짜리 test data를 train window에 담고, 기존의 맨 첫 train data는 배제.
(Walking Forward Analysis)

6. 3-5를 계속 반복

7. 모든 return을 계산했다면, csv파일로 변환해 엑셀에서 KOSPI에 비해 어느 성능을 내는지 비교해보기.
```
<br>
&nbsp;

# 1. 환경 설정 및 데이터 불러오기

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers


drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Google Drive where you have saved the unzipped
FOLDERNAME =  'IFE/'

assert FOLDERNAME is not None, 'ERROR'

%cd drive/My\ Drive
%cp -r $FOLDERNAME ../../


Mounted at /content/drive
/content/drive/My Drive


In [ ]:
# 사용할 optimization technique, quadratic 문제를 풀 수 있다
!pip install cvxopt

# 분산 추정에 사용할 arch library
!pip install arch

In [ ]:
# 모델 수립을 위한 Train/Validation Set, 에러날시 runtime 해제 후 다시 mount
df = pd.read_csv('./IFE/dataset.csv')

In [ ]:

# 사용할 거래 데이터
df

## 2. 포트폴리오 구현

In [ ]:
### 누적 수익률 상위 00개 종목들을 뽑아서, 포트폴리오 구성에 활용할 것임.

adjusted_df = df
adjusted_df = adjusted_df.drop("Symbol", axis = 1)
adjusted_df = adjusted_df.drop(0)

# object 타입의 가격값들이니, 계산을 하기 위해 모든 데이터들의 타입을 int64 자료형으로 변환
for i in adjusted_df.columns:
  adjusted_df[i] = adjusted_df[i].str.replace(',', '').astype('int64')

In [ ]:
## 일단 10개는 시가총액이 제일 높은 거부터 10개로 투자를 해볼 것이다.(2010년 1월에 mcap 데이터를 조회해 ticker를 뽑았음) -> KRX KOSPI에서 따온 데이터
big_mcap_assets = ["A005930", "A005490", "A005380", "A015760", "A003550", "A012330", "A000660", "A017670", "A030200", "A000810"]
# big_mcap_assets = []
selected_df = pd.DataFrame() # 선택한 데이터들을 담는 최종 데이터프레임을 만든다.
use_df = df.drop(0, axis = 0)
selected_df["Date"] = use_df["Symbol"] # 날짜 데이터를 추가
use_df = use_df.drop("Symbol", axis = 1)

for i in big_mcap_assets:
  selected_df[i] = adjusted_df[i]

adjusted_df = adjusted_df.drop(big_mcap_assets, axis = 1)

In [ ]:
# pi(1 + rp)가 큰 상위 00 - 10개의 종목을 구하기 위해 return을 계산 -> 총 00개의 종목을 선정해서 포트폴리오를 구성해볼 것임.
day_return = adjusted_df.pct_change().dropna()

day_return = day_return.reset_index(drop = True)

day_return = day_return + 1

day_return

In [ ]:
cumulative_return = [] # 1 + rp값을 모두 곱한 값을 각 인덱스에 저장

for i in day_return.columns:
  val = 1
  for j in range(0, day_return.shape[0]):
    val *= day_return.loc[j, i]

  cumulative_return.append(val)

In [ ]:
select_asset_index = np.argsort(cumulative_return)[::-1][:40] # 값을 기준으로 내림차순으로 정렬

# 선정한 00 - 10개의 종목들
select_asset_index

In [ ]:
# 선정한 모든 종목들의 주가 값을 저장한다.
for i in select_asset_index:
  col = adjusted_df.columns[i]
  selected_df[col] = adjusted_df[col]

in_sample = [1, 120] ## train
out_sample = [121, selected_df.shape[0]] ## test

selected_df # 이 데이터로 포트폴리오를 구성하며, 매 time-frame을 옮겨가며 rebalancing(solving Markowitz model with no-shortselling, in every iteration)

In [ ]:
## 모든 데이터에 대한 값을 구해놓고, 이를 계속 재활용하는 식으로 진행
def calculate_statistics(data): # 여기서의 data arg.는 사전에 모두 구해놓은 return data
  import numpy as np
  # data = data.drop("Date", axis = 1)
  # returns = data.pct_change().dropna()
  returns = pd.DataFrame(data)
  mean_returns = returns.mean()
  cov_matrix = returns.cov()
  corr_matrix = returns.corr()

  return np.array(returns), np.array(mean_returns), np.array(cov_matrix), np.array(corr_matrix)

In [ ]:
# 전체 데이터에 대한 rate of return을 계산하여 반환해주는 함수
def calculate_all_return(data):
  data = data.drop("Date", axis = 1)
  returns = data.pct_change().dropna()

  return returns

In [ ]:
## GARCH(1,1)으로 분산을 estimate하여, 과거의 상관계수를 함께 활용해 공분산 행렬을 구축하기 위해 구현한 함수
## 이에 대한 정상성 검증 없이 항상 만족을 한다 가정하고 구현을 진행했음.
def garch(returns, mat):
  from arch.univariate import ConstantMean, GARCH, Normal
  from arch import arch_model
  import math

  for i in range(0, returns.shape[1]):
    am = arch_model(returns[:, i], vol = "Garch", p = 1, o = 0, q = 1, dist = "Normal")
    result = am.fit(update_freq = 5)

    forecasts = result.forecast(horizon = 1, reindex = False)

    sigma = math.sqrt(forecasts.variance.iloc[-1, :].values)

    for j in range(0, returns.shape[1]):
        mat[i, j] *= sigma
        mat[j, i] *= sigma

  return mat # 추정값을 구해 만든 공분산 행렬

## rate of return을 추정하기 위한 arima 함수 구현
def arima(returns, mean):
  import statsmodels.api as sm
  from statsmodels.tsa.arima_model import ARIMA

  returns = pd.DataFrame(returns)
  print(returns[0])
  for i in range(0, returns.shape[1]):
    am = sm.tsa.arima.ARIMA(returns[i].diff(), order = (1, 0, 1)).fit()
    forecasts = am.forecast(steps = 1)

    mean[i] = forecasts

  return mean # r에 대한 추정값을 담아서 반환


In [ ]:
## 최소 분산 포트폴리오의 weight를 반환해주는 함수

def Minimum_Variance_Portfolio(data, r_min, start, end, prv):

  n = len(data.columns) - 1 ## 선택한 종목들의 개수

  returns = returns_of_data[start : end]
  returns, mean, P, corr_matrix = calculate_statistics(returns_of_data[start : end]) ## train sample 에 해당하는 row를 과거 데이터로 사용
  P = garch(returns, corr_matrix) ## 기존의 상관계수를 이용하고, 분산을 estimate해서 곱하여 covariance 행렬 구축 -> P
  mean = arima(returns, mean)

  P = cvxopt_matrix(P)
  G = cvxopt_matrix(np.vstack((-np.transpose(mean), -np.eye(n))))
  h = cvxopt_matrix(np.vstack((-np.ones((1,1))*r_min, -np.zeros((n,1)))))
  A = cvxopt_matrix(1.0, (1, n))
  b = cvxopt_matrix(1.0)
  q = cvxopt_matrix(np.zeros((n, 1)))

  # sol = cvxopt_solvers.qp(P, q, G, h, A, b)
  try:
    sol = cvxopt_solvers.qp(P, q, G, h, A, b)

  except: # 만약 답이 발산한다면, terminate하고 전 달의 weight값을 이용한다는 logic 활용
    return prv, P

  weight = []
  for i in sol['x']:
    weight.append(i)

  return weight, P ## 가중치 값을 리스트 형태로 ticker 순서대로 저장해 반환, 최대 샤프 지수 포트폴리오 구할 때 P를 사용할 것이니 이것 역시 반환해주기


In [ ]:
## 최대 샤프 지수 포트폴리오를 만들고 이의 가중치 집합(포트폴리오)을 반환
## scipy 활용
import numpy as np
from scipy import optimize

def Maximum_Sharpe_Ratio_Portfolio(mean, cov, rf, psize):
    weight = []

    def opt_func(x, Mean_Returns, Cov_Returns, RiskFree_Rate, Portfolio_Size):
        Denom = np.sqrt(np.matmul(np.matmul(x, cov), x.T) )
        Nume = np.matmul(np.array(mean),x.T) - rf
        func = -(Nume / Denom)
        return func

    # 제약조건을 담는 함수
    def constraintEq(x):
        A = np.ones(x.shape)
        b = 1
        constraintVal = np.matmul(A, x.T) - b
        return constraintVal

  # 가중치의 제약을 담는 logic
    xinit = np.repeat(0.33, psize)
    cons = ({'type': 'eq', 'fun' : constraintEq})
    lb = 0 ## 가중치의 하한, 상한을 설정
    ub = 1
    bnds = tuple([(lb,ub) for x in xinit])

    #minimize solver(scipy optimizer의 minimize를 사용함 (음수로 만들어서))
    opt = optimize.minimize (opt_func, x0 = xinit, args = (mean, cov,\
                             rf, psize), method = 'SLSQP',  \
                             bounds = bnds, constraints = cons, tol = 10**-3)

    for i in opt['x']:
      weight.append(i)

    return weight

In [ ]:
## Memoization하여서 전체 데이터에 대한 return을 재활용 -> 계산량을 줄이기 위해
returns_of_data = calculate_all_return(selected_df) # 전체 데이터의 return
returns_of_data = returns_of_data.reset_index(drop = True)

In [ ]:
returns_of_data ## 전체 데이터에 대한 rate of return의 dataframe

In [ ]:
## minimum variance 포트폴리오와 maximum sharpe ratio portfolio를 모두 구하되, 백테스팅을 달별로 진행하며, time window를 매번 움직이면서 새로운 weight를 계산하여 return을 테스팅
## walk forward analysis for inspect portfolio weights

idx = 0 # time-frame을 매 iteration마다 옮기기 위한 인덱스
min_ret_list = [] # real data에 대해서 minimum variance portfolio r_p를 계산하고 저장
max_ret_list = [] # real data에 대해서 maximum sharpe ratio portfolio r_p를 계산하고 저장
n = len(selected_df.columns) - 1
prv_weight = [(1 / n)] * n

## 범위를 이유로 마지막 test 구간은 제외하고 구현
for i in range(out_sample[0], out_sample[1] - 1):
  train = [0 + idx, in_sample[1] + idx]
  test = i
  idx += 1

  # 최소 분산 포트폴리오 구하기
  mv_weight_list, cov_matrix = Minimum_Variance_Portfolio(selected_df, 0, train[0], train[1], prv_weight)
  prv_weight = mv_weight_list # 기존 weight를 저장
  # test data에서 수익률이 어느정도 나오는지 확인
  ret = returns_of_data.iloc[test]
  ans = 0
  for j in range(0, n):
    ans += (ret[j] * mv_weight_list[j])

  min_ret_list.append(ans)


  # 최대 샤프 지수 포트폴리오 구하기
  m = calculate_statistics(returns_of_data[train[0] : train[1]])[1]
  mx_weight_list = Maximum_Sharpe_Ratio_Portfolio(m, cov_matrix, 0, n)

  ans = 0

  for j in range(0, n):
    ans += (ret[j] * mx_weight_list[j])

  max_ret_list.append(ans)

  # break ## for debugging, break segmentation invoked

## 3. Return plotting and result analysis

In [ ]:
# backtesting해서 계산한 수익들을 plotting을 해보기, time frame을 월 단위로 잘라서 line plot을 그렸음
import matplotlib.pyplot as plt
import datetime

result_min_df = pd.DataFrame()
result_min_df['Date'] = selected_df.iloc[out_sample[0] : out_sample[1] - 1, 0]
result_min_df = result_min_df.reset_index(drop = True)
result_min_df['rate of return'] = pd.DataFrame(min_ret_list) ## input list를 수정하여 min, max portfolio에 대한 line plot 및 dataframe 생성 가능

result_max_df = pd.DataFrame()
result_max_df['Date'] = selected_df.iloc[out_sample[0] : out_sample[1] - 1, 0]
result_max_df = result_max_df.reset_index(drop = True)
result_max_df['rate of return'] = pd.DataFrame(max_ret_list) ## input list를 수정하여 min, max portfolio에 대한 line plot 및 dataframe 생성 가능

start = datetime.date(2010, 1, 29)
end = datetime.date(2020, 12, 30)

plt.figure(figsize = (20, 5))
plt.plot(result_min_df['Date'], result_min_df['rate of return'], label = 'Minimum_vaiance_portfolio')
plt.plot(result_max_df['Date'], result_max_df['rate of return'], label = 'Maximum Sharpe Ratio Portfolio')
plt.legend()
plt.show()

In [ ]:
# Excel 파일 구성을 위한 return에 대한 csv파일 생성 체크
pd.options.display.max_rows = 200
result_min_df.to_csv('./IFE/result_min_df4.csv') # 현재 min
result_max_df.to_csv('./IFE/result_max_df4.csv')

In [ ]:
returns_of_data.columns # 선정한 ticker

In [ ]:
# 50개 최적, sheet 3
# ['A005930', 'A005490', 'A005380', 'A015760', 'A003550', 'A012330',
#        'A000660', 'A017670', 'A030200', 'A000810', 'A000670', 'A001800',
#        'A003530', 'A010120', 'A001440', 'A024090', 'A008930', 'A002350',
#        'A007340', 'A016360', 'A017800', 'A013120', 'A000490', 'A000240',
#        'A002840', 'A010060', 'A012860', 'A003030', 'A028050', 'A024110',
#        'A034810', 'A001040', 'A005070', 'A014470', 'A006110', 'A017960',
#        'A009520', 'A005950', 'A004430', 'A006730', 'A001380', 'A007280',
#        'A003540', 'A013700', 'A014790', 'A009420', 'A024940', 'A034120',
#        'A010100', 'A023770']

# 60개 최적, sheet 4
# ['A005930', 'A005490', 'A005380', 'A015760', 'A003550', 'A012330',
#        'A000660', 'A017670', 'A030200', 'A000810', 'A019170', 'A002840',
#        'A005690', 'A008930', 'A009420', 'A025950', 'A026960', 'A007700',
#        'A025980', 'A002790', 'A007310', 'A011000', 'A003220', 'A004490',
#        'A003850', 'A006040', 'A033920', 'A003090', 'A011780', 'A000100',
#        'A024720', 'A014680', 'A021240', 'A002620', 'A006280', 'A006730',
#        'A010780', 'A008350', 'A001820', 'A033270', 'A001840', 'A024060',
#        'A009450', 'A011390', 'A012790', 'A025900', 'A005740', 'A003000',
#        'A006060', 'A001070', 'A036490', 'A016250', 'A003350', 'A006890',
#        'A000060', 'A011170', 'A030960', 'A003780', 'A033540', 'A018880']
